In [73]:
import numpy as np
import tensorflow as tf
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [74]:
from tensorflow.keras.layers import Input, Dense, Activation, LeakyReLU, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [75]:
NOISE_DIM = 50
INPUT_DIM = 29
LATENT_DIM = 100

BATCH_SIZE = 100
EPOCHS = 10

In [76]:
discriminator = Sequential(
    [
    InputLayer(input_shape=INPUT_DIM),
    Dense(units=100, kernel_initializer='glorot_normal', activation='relu'),
    Dense(units=50, kernel_initializer='glorot_normal', activation='relu'),
    Dense(units=2, activation='softmax')
    ], 
    name='descriminator'
)

In [77]:
generator = Sequential(
    [
    InputLayer(input_shape=NOISE_DIM),
    Dense(units=LATENT_DIM, kernel_initializer='glorot_normal'),
    Dense(units=INPUT_DIM, activation='softmax'),
    ],
    name='generator'
)

In [78]:
print(f'{discriminator.summary()}\n\n{generator.summary()}')

Model: "descriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               3000      
_________________________________________________________________
dense_16 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 102       
Total params: 8,152
Trainable params: 8,152
Non-trainable params: 0
_________________________________________________________________
Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_19 (Dense)             (None, 29)                2929      
Total params: 8,029
Trainable pa

In [79]:
def discriminator_loss(d_real, d_fake, metrics='JSD'):
  if metrics in ['JSD', 'jsd']:
      real_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(np.zeros(d_real.shape[0]),depth=2),logits=d_real))
      fake_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(np.ones(d_fake.shape[0]),depth=2),logits=d_fake))
      return real_loss + fake_loss
  else:
      raise ValueError

def generator_loss(d_fake, metrics='JSD'):
  if metrics in ['JSD', 'jsd']:
      return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(np.ones(d_fake.shape[0]),depth=2),logits=d_fake))
  else:
      raise ValueError

In [80]:
class GAN(Model):
    def __init__(self, discriminator, generator):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.noise_dim = NOISE_DIM

    def compile(self, d_optimizer, g_optimizer):
        super(GAN, self).compile()
        self.opt_d = d_optimizer
        self.opt_g = g_optimizer
    
    def train_step(self, train_dataset):
        if isinstance(train_dataset, tuple):
            x_batch_train = train_dataset[0]
        batch_size = tf.shape(x_batch_train)[0]

        # Train discriminator
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.noise_dim))
        generated_data = self.generator(random_latent_vectors)
        with tf.GradientTape() as tape:
            d_real = self.discriminator(x_batch_train)
            d_fake = self.discriminator(generated_data)
            
            loss_d = discriminator_loss(d_real, d_fake, 'JSD')
        grads = tape.gradient(loss_d, self.discriminator.trainable_variables)
        self.opt_d.apply_gradients(
            zip(grads, self.discriminator.trainable_variables)
        )

        # Train generator
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.noise_dim))
        with tf.GradientTape() as tape:
            gz = self.generator(random_latent_vectors)
            d_fake = self.discriminator(gz)
            loss_g = generator_loss(d_fake, 'JSD')
        grads = tape.gradient(loss_g, self.generator.trainable_variables)
        self.opt_g.apply_gradients(
            zip(grads, self.generator.trainable_variables)
        )
        return {"loss_d": loss_d, "loss_g": loss_g}

In [81]:
gan = GAN(discriminator=discriminator, generator=generator)
gan.compile(d_optimizer=Adam(learning_rate=0.0003), g_optimizer=Adam(learning_rate=0.001))

In [82]:
n_samples = 200000
n_features = 29
samples = make_classification(n_samples, n_features)

In [83]:
train_x, test_x, train_y, test_y = train_test_split(samples[0], samples[1], test_size=0.2, random_state=123)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.1, random_state=123)

In [84]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
validation_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))

train_dataset = train_dataset.shuffle(buffer_size=512).batch(BATCH_SIZE)

In [85]:
train_loss = gan.fit(train_dataset, epochs=EPOCHS)

Epoch 1/10
1440/1440 [==============================] - 3s 1ms/step - loss_d: 1.0409 - loss_g: 0.7189
Epoch 2/10
1440/1440 [==============================] - 2s 1ms/step - loss_d: 0.6947 - loss_g: 0.3812
Epoch 3/10
1440/1440 [==============================] - 2s 1ms/step - loss_d: 0.6303 - loss_g: 0.3171
Epoch 4/10
1440/1440 [==============================] - 2s 1ms/step - loss_d: 0.6275 - loss_g: 0.3142
Epoch 5/10
1440/1440 [==============================] - 2s 1ms/step - loss_d: 0.6268 - loss_g: 0.3136
Epoch 6/10
1440/1440 [==============================] - 2s 1ms/step - loss_d: 0.6266 - loss_g: 0.3134
Epoch 7/10
1440/1440 [==============================] - 2s 1ms/step - loss_d: 0.6266 - loss_g: 0.3133
Epoch 8/10
1440/1440 [==============================] - 2s 1ms/step - loss_d: 0.6265 - loss_g: 0.3133
Epoch 9/10
1440/1440 [==============================] - 2s 1ms/step - loss_d: 0.6265 - loss_g: 0.3133
Epoch 10/10
1440/1440 [==============================] - 2s 1ms/step - loss_d: 0.6